In [2]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import random
import math
import json
import os
from pathlib import Path
import time
import glob
#LINK_PROJECT = Path(os.path.abspath(__file__)).parent.parent
from numpyencoder import NumpyEncoder

In [14]:
def import_data(input_file):
    with open(input_file) as f:
        data = json.load(f)
        return data
    
def export(output_file, solution):
    if len(solution) == 3:
        with open(output_file, "w+") as f:
            json.dump({"Time": solution["time"], "Result": solution["obj"],
                      "Solution": solution["field"]}, f, cls=NumpyEncoder)
    else:
        with open(output_file, "w+") as f:
            json.dump({"Time": solution["time"], "Result": solution["obj"]}, f, cls=NumpyEncoder)
    
def process(input_file, output_file, time_limit):
    data = import_data(input_file)
    solution = Greedy(data)
    export(output_file, solution)

In [16]:
#data = import_data('../data/data_v2/HomologousField/sample_1000_10_50.json')
def Greedy(data):
    # Opening JSON file
    N = data['N']; m = data['m']; M = data['M']; d = data['d']; s = data['s']; e = data['e']
    fields = list(zip(d, s, e))
    
    start_time = time.time()
    solution = {}
    fields = sorted(fields, key=lambda x: (x[1], x[2]))
    total_quantity = sum(d)
    avg_quantity = total_quantity / N
#     target = round((m + M) / 2 / avg_quantity) * avg_quantity
    target = (m + M) / 2
    num_days = int(total_quantity / target)
    num_fields_in_low_target_days = N // num_days
    num_fields_in_high_target_days = num_fields_in_low_target_days + 1
    num_high_target_days = N - num_fields_in_low_target_days * num_days
    num_low_target_days = num_days - num_high_target_days
#     print(N)
#     print(target)
#     print(num_days)
#     print(num_fields_in_low_target_days)
#     print(num_fields_in_high_target_days)
#     print(num_low_target_days)
#     print(num_high_target_days)
#     last_day_to_harvest = 0
#     strategy = list()
#     strategy.append(list())
    last_day_to_harvest = 0
    field_index = 0
    results = list()
    quantities = list()
    for day in range(num_low_target_days):
        total_quantity = 0
        end_days = list()
        for i in range(num_fields_in_low_target_days):
            field = fields[field_index]
            total_quantity += field[0]
            end_days.append(field[2])
            field_index += 1;
        day_to_harvest = max(last_day_to_harvest, field[1])
        last_day_to_harvest += 1
        if day_to_harvest > min(end_days):
            solution["time"] = time.time() - start_time
            solution["obj"] = "No Solution"
            return solution
        if total_quantity < m or total_quantity > M:
            solution["time"] = time.time() - start_time
            solution["obj"] = "No Solution"
            return solution
        for i in range(num_fields_in_low_target_days):
            results.append(day_to_harvest)
        quantities.append(total_quantity)
            
    for day in range(num_high_target_days):
        total_quantity = 0
        end_days = list()
        for i in range(num_fields_in_high_target_days):
            field = fields[field_index]
            total_quantity += field[0]
            end_days.append(field[2])
            field_index += 1;
        day_to_harvest = max(last_day_to_harvest, field[1])
        last_day_to_harvest += 1
        if day_to_harvest > min(end_days):
            solution["time"] = time.time() - start_time
            solution["obj"] = "No Solution"
            return solution
        if total_quantity < m or total_quantity > M:
            solution["time"] = time.time() - start_time
            solution["obj"] = "No Solution"
            return solution
        for i in range(num_fields_in_high_target_days):
            results.append(day_to_harvest)
        quantities.append(total_quantity)
            
                
    solution["time"] = time.time() - start_time
    solution["obj"] = max(quantities) - min(quantities)
    solution["field"] = results   
    return solution

#Greedy(data)

In [18]:
for path in glob.glob("..\\data\\data_v2\\**\\**.json"):
    datatype, name = path.split("\\")[-2:]
    print("Type: ", datatype, " | Name: ", name)
    input_file = f"../data/data_v2/{datatype}/{name}"
    output_file = f"../results/data_v2/{datatype}/greedy/result_{name}"
    time_limit = 600000
    process(input_file, output_file, time_limit)

Type:  HomologousField  | Name:  sample_1000_10_50.json
Type:  HomologousField  | Name:  sample_1000_40_75.json
Type:  HomologousField  | Name:  sample_1000_50_100.json
Type:  HomologousField  | Name:  sample_2000_10_50.json
Type:  HomologousField  | Name:  sample_2000_50_100.json
Type:  HomologousField  | Name:  sample_500_10_50.json
Type:  Type1Large  | Name:  sample_1000_20_30.json
Type:  Type1Large  | Name:  sample_1000_30_50.json
Type:  Type1Large  | Name:  sample_1000_40_75.json
Type:  Type1Large  | Name:  sample_1000_50_100.json
Type:  Type1Large  | Name:  sample_1000_60_125.json
Type:  Type1Large  | Name:  sample_100_20_30.json
Type:  Type1Large  | Name:  sample_100_30_50.json
Type:  Type1Large  | Name:  sample_100_40_75.json
Type:  Type1Large  | Name:  sample_100_50_100.json
Type:  Type1Large  | Name:  sample_100_60_125.json
Type:  Type1Large  | Name:  sample_250_20_30.json
Type:  Type1Large  | Name:  sample_250_30_50.json
Type:  Type1Large  | Name:  sample_250_40_75.json
Type

In [23]:
for path in glob.glob("..\\data\\data_v2\\HomologousField\\*2000_50_100*.json"):
    datatype, name = path.split("\\")[-2:]
    print("Type: ", datatype, " | Name: ", name)
    input_file = f"../data/data_v2/{datatype}/{name}"
    output_file = f"../results/data_v2/{datatype}/greedy/result_{name}"
    time_limit = 600000
    process(input_file, output_file, time_limit)


Type:  HomologousField  | Name:  sample_2000_50_100.json


In [22]:
5.6 / 2.1

2.6666666666666665

In [26]:
list()

[]